# Disk Stars

In [1]:
from typing import Optional, Tuple
from astropy.io import fits
from astropy.table import Table, vstack
import numpy as np
import pandas as pd
import glob
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde
import matplotlib.cm as cm
import seaborn as sns
sns.set_theme(style="ticks")

In [2]:
# Open the FITS files
# Define the pattern to match FITS files
pattern = '../Disk_Lomeli/*.fits'

# Use glob to find all FITS files in the current directory
fits_files = glob.glob(pattern)



In [3]:
# Create an empty list to store individual tables
tables = []
    

In [ ]:
# Loop through each FITS file and read it into a Table
for file in fits_files:
    with fits.open(file) as hdul:
        # Extract data from the FITS file
        data = hdul[1].data  # Assuming the data is in the first extension (HDU index 1)
        
        # Convert the data to a Table
        table = Table(data)
        
        # Append the Table to the list
        tables.append(table)

In [ ]:
# Concatenate all Tables into a single Table
final_table_ = vstack(tables)
final_table_

In [ ]:
# Define chunk size
chunk_size = 10000  # Adjust this value as needed

# Create an empty list to store chunks of filtered tables
filtered_chunks = []

# Loop over chunks of the table
for chunk_start in range(0, len(final_table_), chunk_size):
    chunk_end = min(chunk_start + chunk_size, len(final_table_))
    chunk = final_table_[chunk_start:chunk_end]

    # Apply filtering directly to the chunk
    filtered_chunk = chunk[(chunk["field"] != "d049") & (chunk["field"] != "d203")]

    # Append filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate filtered chunks into a single table
final_table = Table.vstack(filtered_chunks)

### Cleaning

In [ ]:
final_table.colnames


In [ ]:
# Masking, usar FLATS menor que 4
# Masking
m_x =  (final_table["r"] >= 13) & (final_table["r"] <= 19.5) &\
       (final_table["F660"] <= 19.4) & (final_table["i"] <= 19.2) 

m_err = (final_table["rerr"] <= 0.2) &  \
        (final_table["F660err"] <= 0.2) & \
        (final_table["ierr"] <= 0.2)


#m_sn = (tab["s2n_r_PStotal"] >= 10) & \
#       (tab["s2n_J0660_PStotal"] >= 10) & \
#        (tab["s2n_i_PStotal"] >= 10)

m_ell = (final_table["ellip_r"] <= 0.2) & (final_table["ellip_i"] <= 0.2)

m_50 = (final_table["flux_radius_r"] <= 3) & (final_table["flux_radius_i"] <= 3)

flags = (final_table["flags_r"] == 0) & (final_table["flags_i"] == 0)

class_star = (final_table["class_r"] > 0.5) & (final_table["class_i"] > 0.5)

mask = m_x & m_err & m_ell & m_50 & flags & class_star# Use fwhm_mask instead of valid_fwhm_range
tab_clean = final_table[mask]


In [ ]:
tab_clean

In [ ]:
ri = tab_clean["r"] - tab_clean["i"]
rj660 = tab_clean["r"] - tab_clean["F660"]

In [ ]:
def equation(m, c, x):
    # Define your equation here, for example, a linear equation: y = mx + c
    y = m * x + c
    return y

In [ ]:
def plots(
    color1: np.ndarray,
    color2: np.ndarray,
    x_values: np.ndarray,
    y_values: np.ndarray,
    y_values1: Optional[np.ndarray] = None,
    sample_size: Optional[int] = None,
    xlim: Optional[Tuple[float, float]] = None,
    ylim: Optional[Tuple[float, float]] = None,
    density_method: str = "gaussian_kde",
    colormap: str = "viridis",
    label1: str = None,
    label2: str = None,
    label3: str = None,
    title: str = "Scatter Plot with Density Coloring"
) -> None:
    """
    Plots a scatter plot with points colored based on their density and one or two lines 
    defined by x_values, y_values, and optionally y_values1.
    """

    # Handle sampling and duplicate colors, ensuring equal lengths
    if sample_size is not None:
        unique_indices = np.random.choice(len(color1), size=sample_size, replace=False)
        color1, color2 = color1[unique_indices], color2[unique_indices]

    # Calculate density
    xy = np.vstack([color1, color2])
    if density_method == "gaussian_kde":
        density = gaussian_kde(xy)(xy)
    else:
        raise NotImplementedError(f"Density method {density_method} not supported.")

    # Sort points by density
    sorted_indices = density.argsort()[::-1]  # Descending order
    color1, color2, density = color1[sorted_indices], color2[sorted_indices], density[sorted_indices]

    # Create plot
    fig, ax = plt.subplots(figsize=(14, 11))

    # Customize scatter plot
    scatter_collection = ax.scatter(
        color1,
        color2,
        c=density,
        s=10,
        cmap=colormap,
        alpha=0.8,
        label=label1 if label1 else "Color 1"
    )

    # Customize colorbar
    plt.colorbar(scatter_collection, label="Density")

    # Customize line plots
    plt.plot(x_values, y_values, linewidth=2, color="red", label=label2 if label2 else "Line 1")
    if y_values1 is not None:
        plt.plot(x_values, y_values1, linewidth=2, color="blue", label=label3 if label3 else "Line 2")

    # Set limits
    if xlim is not None:
        plt.xlim(xlim)
    if ylim is not None:
        plt.ylim(ylim)

    # Add labels and title
    plt.xlabel(label1 if label1 else "Color 1")
    plt.ylabel(label2 if label2 else "Color 2")
    plt.title(title)

    # Add legend if labels are provided
    if label1 or label2 or label3:
        plt.legend()

    plt.show()

In [ ]:
# Generate x values
x_values = np.linspace(-10, 10, 100)  # Generate 100 x values from 0 to 10

# Compute y values using the equation
y_values = equation(0.35, -0.55, x_values)

In [ ]:
y_values1 = equation(0.35, 0.45, x_values)

In [ ]:
plots(ri, rj660, x_values, y_values, y_values1, xlim=(-1.5, 3), ylim=(-5, 4), sample_size=10000)

In [ ]:
# Select objects below the line
# below_line = rj660 < equation(ri)
color_crit = (rj660  <= 0.35*ri - 0.55)

# upper_line = rj660 < equation(ri)
color_crit_upper = (rj660 >= 0.35*ri + 0.45)

In [ ]:
tab_clean_below = tab_clean[color_crit]
len(tab_clean_below)

In [ ]:
ri_below = tab_clean_below["r"] - tab_clean_below["i"]
rj660_below = tab_clean_below["r"] - tab_clean_below["F660"]



plots(ri_below, rj660_below, x_values, y_values, xlim=(-1.5, 3), ylim=(-5, 4))

In [ ]:
df_bellow = tab_clean_below.to_pandas()
#Select columns 
df_bellow_ = df_bellow[["ALPHA", "DELTA", "x", "y", "fwhm_r", "r", "F660", "i"]]
             
sns.pairplot(df_bellow_)

### Upper

In [ ]:
tab_clean_upper = tab_clean[color_crit_upper]
tab_clean_upper

In [ ]:
ri_upper = tab_clean_upper["r"] - tab_clean_upper["i"]
rj660_upper = tab_clean_upper["r"] - tab_clean_upper["F660"]


plots(ri_upper, rj660_upper, x_values, y_values1, xlim=(-1.5, 3), ylim=(-5, 4))

In [ ]:
df_bellow = tab_clean_upper.to_pandas()
#Select columns 
df_bellow_ = df_bellow[["ALPHA", "DELTA", "x", "y", "fwhm_r", "r", "F660", "i"]]
             
sns.pairplot(df_bellow_)

# Spliting by r bin

In [ ]:
# Define conditions for splitting the table based on the "r" column
condition_1 = tab_clean['r'] <= 16
condition_2 = (tab_clean['r'] > 16) & (tab_clean['r'] <= 17.5)
condition_3 = (tab_clean['r'] > 17.5) & (tab_clean['r'] <= 18.5)
condition_4 = (tab_clean['r'] > 18.5) & (tab_clean['r'] <= 19.5)

# Apply conditions to filter the table
table_1 = tab_clean[condition_1]
table_2 = tab_clean[condition_2]
table_3 = tab_clean[condition_3]
table_4 = tab_clean[condition_4]

In [ ]:
len(table_1)

In [ ]:
# Convert Astropy tables to pandas DataFrames and save each one
df_table_1 = table_1.to_pandas()
df_table_1.to_csv('../Disk_Lomeli/disk_splus_13r16.csv', index=False)
del df_table_1  # Free memory by deleting the DataFrame

In [ ]:
df_table_2 = table_2.to_pandas()
df_table_2.to_csv('../Disk_Lomeli/disk_splus_16r175.csv', index=False)
del df_table_2

In [ ]:
df_table_3 = table_3.to_pandas()
df_table_3.to_csv('../Disk_Lomeli/disk_splus_175r185.csv', index=False)
del df_table_3

In [ ]:
df_table_4 = table_4.to_pandas()
df_table_4.to_csv('../Disk_Lomeli/disk_splus_185r195.csv', index=False)
del df_table_4

In [ ]:
print("CSV files for each table saved successfully.")